In [9]:
import httpx

In [1]:
LANGFUSE_PUBLIC_KEY = "pk-lf-f6ea7cfd-fb41-4ee5-813e-8152763a5762"
LANGFUSE_SECRET_KEY = "sk-lf-ff29c0d1-50fd-413b-bfb1-136e4782e9d3"

In [ ]:
#Get all annotation queues

import base64

# Create Basic auth header
auth_string = f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}"
auth_bytes = auth_string.encode('ascii')
auth_b64 = base64.b64encode(auth_bytes).decode('ascii')

async with httpx.AsyncClient() as client:
    response = await client.get(
        "https://us.cloud.langfuse.com/api/public/annotation-queues",
        headers={
            "Authorization": f"Basic {auth_b64}"
        },
        params={
            "page": 1,
            "limit": None  # Use None instead of "null" string
        }
    )
    response_json = response.json()

In [18]:
response_json

{'data': [{'id': 'cmccb2ym402tpad07asml1mb7',
   'name': 'test - vishal',
   'description': None,
   'scoreConfigIds': ['cmbwd9hk302riad07n5n2jlvq',
    'cmbhj8097049kad0763m80muh'],
   'createdAt': '2025-06-25T18:46:46.205Z',
   'updatedAt': '2025-08-13T07:58:37.099Z'},
  {'id': 'cmb7albrq0eodad075gmi1vrd',
   'name': 'test-queue-01',
   'description': None,
   'scoreConfigIds': ['cmbhj8097049kad0763m80muh',
    'cmb59k48v00hqad07e7dvolfn'],
   'createdAt': '2025-05-28T01:54:30.230Z',
   'updatedAt': '2025-06-04T06:03:54.831Z'}],
 'meta': {'page': 1, 'limit': 50, 'totalItems': 2, 'totalPages': 1}}

## Get Specific Annotation Queue by ID

In [ ]:
# Function to get a specific annotation queue by ID
async def get_annotation_queue(queue_id):
    """
    Fetch a specific annotation queue by its ID.
    
    Args:
        queue_id (str): The unique identifier of the annotation queue
        
    Returns:
        dict: Annotation queue details including id, name, description, scoreConfigIds, createdAt, updatedAt
    """
    import base64
    
    auth_string = f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}"
    auth_bytes = auth_string.encode('ascii')
    auth_b64 = base64.b64encode(auth_bytes).decode('ascii')
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"https://us.cloud.langfuse.com/api/public/annotation-queues/{queue_id}",
            headers={
                "Authorization": f"Basic {auth_b64}"
            }
        )
        return response.json()

# Example: Get the first queue from the list above
queue_id = "cmccb2ym402tpad07asml1mb7"  # test - vishal queue
queue_details = await get_annotation_queue(queue_id)
print(f"Queue Name: {queue_details.get('name')}")
print(f"Queue ID: {queue_details.get('id')}")
print(f"Score Config IDs: {queue_details.get('scoreConfigIds')}")
print(f"Created: {queue_details.get('createdAt')}")
print(f"Updated: {queue_details.get('updatedAt')}")

Queue Name: test - vishal
Queue ID: cmccb2ym402tpad07asml1mb7
Score Config IDs: ['cmbwd9hk302riad07n5n2jlvq', 'cmbhj8097049kad0763m80muh']
Created: 2025-06-25T18:46:46.205Z
Updated: 2025-08-13T07:58:37.099Z


## Get Annotation Queue Items

In [ ]:
# Function to get items for a specific annotation queue
async def get_annotation_queue_items(queue_id, status=None, page=1, limit=None):
    """
    Get items for a specific annotation queue.
    
    Args:
        queue_id (str): The unique identifier of the annotation queue
        status (str, optional): Filter by status (e.g., "PENDING", "COMPLETED")
        page (int): Page number, starts at 1 (default: 1)
        limit (int, optional): Number of items per page
        
    Returns:
        dict: Paginated response with queue items including:
            - data: Array of AnnotationQueueItems (id, queueId, objectId, objectType, status, completedAt, createdAt, updatedAt)
            - meta: Pagination metadata (page, limit, totalItems, totalPages)
    """
    import base64
    
    auth_string = f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}"
    auth_bytes = auth_string.encode('ascii')
    auth_b64 = base64.b64encode(auth_bytes).decode('ascii')
    
    # Build query parameters
    params = {"page": page}
    if status:
        params["status"] = status
    if limit:
        params["limit"] = limit
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"https://us.cloud.langfuse.com/api/public/annotation-queues/{queue_id}/items",
            headers={
                "Authorization": f"Basic {auth_b64}"
            },
            params=params
        )
        return response.json()

# Example: Get pending items from the test-vishal queue
queue_id = "cmccb2ym402tpad07asml1mb7"  # test - vishal queue
items = await get_annotation_queue_items(queue_id, status=None, page=1, limit=10)

print(f"Total items: {items['meta']['totalItems']}")
print(f"Total pages: {items['meta']['totalPages']}")
print(f"\nFirst {len(items['data'])} items:")
for item in items['data']:
    print(f"  - Item ID: {item['id']}")
    print(f"    Object ID: {item['objectId']}")
    print(f"    Object Type: {item['objectType']}")
    print(f"    Status: {item['status']}")
    print(f"    Created: {item['createdAt']}")
    print()

Total items: 51
Total pages: 6

First 10 items:
  - Item ID: cmel01iqn08szad08a89nek6f
    Object ID: 3fbb955f-94d3-43ec-80a9-d893e761c6ab
    Object Type: SESSION
    Status: PENDING
    Created: 2025-08-21T06:07:03.454Z

  - Item ID: cmel01iqn08syad08vv7tysps
    Object ID: ebda2d2a-30df-4bf6-a2f1-4f61e859968e
    Object Type: SESSION
    Status: PENDING
    Created: 2025-08-21T06:07:03.454Z

  - Item ID: cmel01iqn08sxad080vpzh1pn
    Object ID: 4519bbb8-e78c-4084-a902-a83b496efb24
    Object Type: SESSION
    Status: PENDING
    Created: 2025-08-21T06:07:03.454Z

  - Item ID: cmel01iqn08swad08swqvnjmg
    Object ID: 847c6285-8fc9-4560-a83f-4e62sss85809254
    Object Type: SESSION
    Status: PENDING
    Created: 2025-08-21T06:07:03.454Z

  - Item ID: cmel01iqn08svad087z2ahc2u
    Object ID: 847c6285-8fc9-4560-a8ss3f-4e6285809254
    Object Type: SESSION
    Status: PENDING
    Created: 2025-08-21T06:07:03.454Z

  - Item ID: cmel01iqn08suad08ouog91f7
    Object ID: 847c6285-8fc9-45d6

In [27]:
# Get ALL items for a specific queue (no status filter)
queue_id = "cmccb2ym402tpad07asml1mb7"  # test - vishal queue

# First, get initial page to see total items
all_items = []
page = 1
limit = 50  # Max items per page

# Get first page to check total
first_page = await get_annotation_queue_items(queue_id, status=None, page=page, limit=limit)
all_items.extend(first_page['data'])
total_items = first_page['meta']['totalItems']
total_pages = first_page['meta']['totalPages']

print(f"Total items in queue: {total_items}")
print(f"Total pages: {total_pages}")

# Get remaining pages if there are more
while page < total_pages:
    page += 1
    next_page = await get_annotation_queue_items(queue_id, status=None, page=page, limit=limit)
    all_items.extend(next_page['data'])
    print(f"Fetched page {page}/{total_pages}")

print(f"\nRetrieved {len(all_items)} items total")

# Display summary by status
from collections import Counter
status_counts = Counter(item['status'] for item in all_items)
print("\nItems by status:")
for status, count in status_counts.items():
    print(f"  {status}: {count}")

# Show first few items as example
print(f"\nFirst 5 items (of {len(all_items)}):")
for item in all_items[:5]:
    print(f"  - Item ID: {item['id']}")
    print(f"    Object ID: {item['objectId']}")
    print(f"    Object Type: {item['objectType']}")
    print(f"    Status: {item['status']}")
    if item.get('completedAt'):
        print(f"    Completed: {item['completedAt']}")
    print()

Total items in queue: 61
Total pages: 2
Fetched page 2/2

Retrieved 61 items total

Items by status:
  PENDING: 51
  COMPLETED: 10

First 5 items (of 61):
  - Item ID: cmel01iqn08szad08a89nek6f
    Object ID: 3fbb955f-94d3-43ec-80a9-d893e761c6ab
    Object Type: SESSION
    Status: PENDING

  - Item ID: cmel01iqn08syad08vv7tysps
    Object ID: ebda2d2a-30df-4bf6-a2f1-4f61e859968e
    Object Type: SESSION
    Status: PENDING

  - Item ID: cmel01iqn08sxad080vpzh1pn
    Object ID: 4519bbb8-e78c-4084-a902-a83b496efb24
    Object Type: SESSION
    Status: PENDING

  - Item ID: cmel01iqn08swad08swqvnjmg
    Object ID: 847c6285-8fc9-4560-a83f-4e62sss85809254
    Object Type: SESSION
    Status: PENDING

  - Item ID: cmel01iqn08svad087z2ahc2u
    Object ID: 847c6285-8fc9-4560-a8ss3f-4e6285809254
    Object Type: SESSION
    Status: PENDING



## Get Specific Queue Item

In [30]:
# Function to get a specific item from an annotation queue
async def get_queue_item(queue_id, item_id):
    """
    Get a specific item from an annotation queue.
    
    Args:
        queue_id (str): The unique identifier of the annotation queue
        item_id (str): The unique identifier of the annotation queue item
        
    Returns:
        dict: Item details including:
            - id: Item ID
            - queueId: Queue ID
            - objectId: ID of the object being annotated
            - objectType: Type of object (e.g., "TRACE")
            - status: Status (e.g., "PENDING", "COMPLETED")
            - completedAt: Completion timestamp (if completed)
            - createdAt: Creation timestamp
            - updatedAt: Last update timestamp
    """
    import base64
    
    auth_string = f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}"
    auth_bytes = auth_string.encode('ascii')
    auth_b64 = base64.b64encode(auth_bytes).decode('ascii')
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"https://us.cloud.langfuse.com/api/public/annotation-queues/{queue_id}/items/{item_id}",
            headers={
                "Authorization": f"Basic {auth_b64}"
            }
        )
        return response.json()

# Example: Get a specific item from the queue
# Use the first item from the all_items list we fetched earlier
if all_items:
    first_item = all_items[1]
    queue_id = "cmccb2ym402tpad07asml1mb7"  # test - vishal queue
    item_id = first_item['id']
    
    item_details = await get_queue_item(queue_id, item_id)
    
    print("Specific Item Details:")
    print(f"  Item ID: {item_details['id']}")
    print(f"  Queue ID: {item_details['queueId']}")
    print(f"  Object ID: {item_details['objectId']}")
    print(f"  Object Type: {item_details['objectType']}")
    print(f"  Status: {item_details['status']}")
    print(f"  Created: {item_details['createdAt']}")
    print(f"  Updated: {item_details['updatedAt']}")
    if item_details.get('completedAt'):
        print(f"  Completed: {item_details['completedAt']}")
else:
    print("No items found in the queue to test with")

Specific Item Details:
  Item ID: cmel01iqn08syad08vv7tysps
  Queue ID: cmccb2ym402tpad07asml1mb7
  Object ID: ebda2d2a-30df-4bf6-a2f1-4f61e859968e
  Object Type: SESSION
  Status: PENDING
  Created: 2025-08-21T06:07:03.454Z
  Updated: 2025-08-21T06:07:03.454Z


## Get Trace Details

In [39]:
# Function to get trace details by trace ID
async def get_trace(trace_id):
    """
    Get a specific trace by its ID.
    
    Args:
        trace_id (str): The unique Langfuse identifier of a trace
        
    Returns:
        dict: Trace details including:
            - id: Trace ID
            - timestamp: Trace timestamp
            - name: Trace name (optional)
            - input/output: Trace input/output data
            - sessionId: Associated session ID
            - userId: Associated user ID
            - metadata: Trace metadata
            - tags: List of tags
            - observations: Array of observations (spans/generations)
            - scores: Array of scores/evaluations
            - totalCost: Total cost of the trace
            - latency: Trace latency in ms
            - environment: Environment (production, staging, etc.)
    """
    import base64
    
    auth_string = f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}"
    auth_bytes = auth_string.encode('ascii')
    auth_b64 = base64.b64encode(auth_bytes).decode('ascii')
    
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"https://us.cloud.langfuse.com/api/public/traces/{trace_id}",
            headers={
                "Authorization": f"Basic {auth_b64}"
            }
        )
        return response.json()

# Example: Get trace details for items in the queue
# First, let's get a trace ID from one of our queue items
if all_items:
    # Find a TRACE type item
    trace_items = [item for item in all_items if item['objectType'] == 'TRACE']
    
    if trace_items:
        trace_item = trace_items[0]
        trace_id = trace_item['objectId']
        
        print(f"Fetching trace: {trace_id}")
        trace_details = await get_trace(trace_id)
        
        print(f"\nTrace Details:")
        print(f"  ID: {trace_details['id']}")
        print(f"  Timestamp: {trace_details['timestamp']}")
        print(f"  Name: {trace_details.get('name', 'N/A')}")
        print(f"  Session ID: {trace_details.get('sessionId', 'N/A')}")
        print(f"  User ID: {trace_details.get('userId', 'N/A')}")
        print(f"  Total Cost: ${trace_details.get('totalCost', 0):.6f}")
        print(f"  Latency: {trace_details.get('latency', 'N/A')} ms")
        print(f"  Environment: {trace_details.get('environment', 'N/A')}")
        
        # Show observations summary
        observations = trace_details.get('observations', [])
        print(f"\n  Observations ({len(observations)} total):")
        for i, obs in enumerate(observations[:3]):  # Show first 3
            print(f"    {i+1}. Type: {obs['type']}, Name: {obs.get('name', 'N/A')}")
            if obs.get('model'):
                print(f"       Model: {obs['model']}")
            if obs.get('usage'):
                usage = obs['usage']
                print(f"       Usage: Input={usage.get('input')}, Output={usage.get('output')}, Cost=${usage.get('totalCost', 0):.6f}")
        
        # Show scores if any
        scores = trace_details.get('scores', [])
        if scores:
            print(f"\n  Scores ({len(scores)} total):")
            for score in scores[:3]:  # Show first 3
                print(f"    - {score['name']}: {score['value']} ({score['dataType']})")
                print(f"      Source: {score['source']}")
        
        # Show tags
        tags = trace_details.get('tags', [])
        if tags:
            print(f"\n  Tags: {', '.join(tags)}")
    else:
        print("No TRACE type items found in the queue")
else:
    print("No items available to get trace from")

Fetching trace: b56b919d-e923-4e63-8b9b-72394b4e2244

Trace Details:
  ID: b56b919d-e923-4e63-8b9b-72394b4e2244
  Timestamp: 2025-08-12T10:27:02.466Z
  Name: LangGraph
  Session ID: f53f9ae7-7ac5-4ff1-b461-6bdc2f3612cf
  User ID: 277aa027-e26b-4bc9-9050-a62e7b887283
  Total Cost: $0.007140
  Latency: 2.878 ms
  Environment: default

  Observations (4 total):
    1. Type: SPAN, Name: agent
       Usage: Input=0, Output=0, Cost=$0.000000
    2. Type: SPAN, Name: LangGraph
       Usage: Input=0, Output=0, Cost=$0.000000
    3. Type: GENERATION, Name: ChatAnthropic
       Model: claude-sonnet-4-20250514
       Usage: Input=2045, Output=67, Cost=$0.000000


In [40]:
trace_details

{'id': 'b56b919d-e923-4e63-8b9b-72394b4e2244',
 'projectId': 'cmaxr5f6h002bad07s9drmhxz',
 'name': 'LangGraph',
 'timestamp': '2025-08-12T10:27:02.466Z',
 'environment': 'default',
 'tags': [],
 'bookmarked': False,
 'release': None,
 'version': None,
 'userId': '277aa027-e26b-4bc9-9050-a62e7b887283',
 'sessionId': 'f53f9ae7-7ac5-4ff1-b461-6bdc2f3612cf',
 'public': False,
 'input': {'messages': [{'content': 'asda',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '808eda1d-290b-493c-8249-c124e23aaaf2',
    'example': False}]},
 'output': {'messages': [{'content': 'asda',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '808eda1d-290b-493c-8249-c124e23aaaf2',
    'example': False},
   {'content': [{'text': "Hello! I'm Stanton, and I'm here to help you understand any legal issues you might be facing. \n\nIt looks like your message might have been sent accidentally. What's g

In [38]:
all_items[-5:]

[{'id': 'cme9mtmij0g1dad07gxjiwchz',
  'queueId': 'cmccb2ym402tpad07asml1mb7',
  'objectId': '25145808-afeb-47d7-b24e-f5de5c215ca6',
  'objectType': 'TRACE',
  'status': 'COMPLETED',
  'completedAt': '2025-08-13T07:12:07.750Z',
  'createdAt': '2025-08-13T07:11:32.156Z',
  'updatedAt': '2025-08-13T07:12:07.751Z'},
 {'id': 'cme9mtmij0g1cad07uawwgz2y',
  'queueId': 'cmccb2ym402tpad07asml1mb7',
  'objectId': '9fdfda4e-bf63-4559-bc99-6c46a3531a91',
  'objectType': 'TRACE',
  'status': 'COMPLETED',
  'completedAt': '2025-08-13T07:12:54.539Z',
  'createdAt': '2025-08-13T07:11:32.156Z',
  'updatedAt': '2025-08-13T07:12:54.540Z'},
 {'id': 'cme9mtmij0g1bad076pvfyoxm',
  'queueId': 'cmccb2ym402tpad07asml1mb7',
  'objectId': 'b4a14450-dd1a-4061-85a3-763993c1e388',
  'objectType': 'TRACE',
  'status': 'COMPLETED',
  'completedAt': '2025-08-13T07:12:39.522Z',
  'createdAt': '2025-08-13T07:11:32.156Z',
  'updatedAt': '2025-08-13T07:12:39.523Z'},
 {'id': 'cme642hky0s80ad07cikpq22k',
  'queueId': 'cmcc